In [ ]:
from tabulate import tabulate

EXAMPLE = "../example.txt"
EXAMPLE_MIN_GAIN = 20

INPUT = "../input.txt"
INPUT_MIN_GAIN = 100

In [ ]:
class Map:
    grid: list[list[str]]
    height: int
    width: int
    start: tuple[int, int]
    end: tuple[int, int]
    moves: list[tuple[int, int]] = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    cheats: dict[tuple[tuple[int, int], tuple[int, int]], int]
    gains: dict[int, int]

    def __init__(self, grid):
        self.grid = grid
        self.height = len(self.grid)
        self.width = len(self.grid[0])
        self.find_start_stop()

    @classmethod
    def build(cls, input_file_name):
        grid = []
        with open(input_file_name, "r") as f:
            for line in f:
                grid.append([c for c in line.replace("\n", "").strip()])
        return cls(grid)

    def find_start_stop(self):
        self.start, self.end = (-1, -1), (-1, -1)
        for row in range(self.height):
            for col in range(self.width):
                if self.grid[row][col] == "S":
                    self.start = (row, col)
                if self.grid[row][col] == "E":
                    self.end = (row, col)
                if self.start != (-1, -1) and self.end != (-1, -1):
                    return

    def next_position(self, position):
        # Find the next position (unique since the track is predetermined)
        row, col = position
        for r, c in self.moves:
            new_row, new_col = row + r, col + c
            if (new_row, new_col) not in self.track:
                if 0 <= new_row < self.height and 0 <= new_col < self.width:
                    if self.grid[new_row][new_col] != "#":
                        return (new_row, new_col)

    def find_race_track(self):
        # Save the positions of the track and the time it takes to reach each one
        position = self.start
        self.track = {}
        time = 0
        while position != self.end:
            self.track[position] = time
            position = self.next_position(position)
            time += 1
        self.track[self.end] = time

    def find_cheats(self, max_nb_of_moves):
        self.cheats = {}
        self.gains = {}
        # Go through all positions of the track, each one could be the start of a cheat
        for cheat_start in self.track:
            nb_of_moves = 1
            # Keep track of the positions we've reached by cheating from this start position in "nb_of_moves" moves
            current_positions = set([cheat_start])
            while nb_of_moves <= max_nb_of_moves:
                new_positions = set()
                # For each current position, calculate the next possible ones
                for row, col in current_positions:
                    for r, c in self.moves:
                        # New possible position
                        cheat_end = (
                            row + r,
                            col + c,
                        )
                        # We can't leave the grid
                        if (
                            row + r < 0
                            or row + r >= self.height
                            or col + r < c
                            or col + c >= self.width
                        ):
                            continue
                        # If the current cheat already exists, no need to pursue it further, we've already explored it
                        if (cheat_start, cheat_end) in self.cheats:
                            continue
                        if cheat_end in self.track:
                            # The new position is on the track so it could be the end of a cheat
                            # Calculate the time gain that cheat would give
                            gain = (
                                self.track[cheat_end]
                                - self.track[cheat_start]
                                - nb_of_moves
                            )
                            # If the gain is a net positive, save the cheat and update the number of cheats for that gain value
                            if gain > 0:
                                self.cheats[(cheat_start, cheat_end)] = gain
                                if gain in self.gains:
                                    self.gains[gain] += 1
                                else:
                                    self.gains[gain] = 1
                        # This new position will be a starting point for the next step
                        new_positions.add(cheat_end)
                nb_of_moves += 1
                current_positions = new_positions

    def count_cheats_above(self, min_gain):
        result = 0
        for gain, nb in self.gains.items():
            if gain >= min_gain:
                result += nb
        return result

In [ ]:
map = Map.build(EXAMPLE)
print(tabulate(map.grid))
map.find_race_track()
print(map.track)

In [ ]:
map.find_cheats(max_nb_of_moves=2)
print(map.cheats)
print(map.gains)

In [ ]:
def part_1(input_file_name, min_gain):
    map = Map.build(input_file_name)
    map.find_race_track()
    map.find_cheats(max_nb_of_moves=2)
    result = map.count_cheats_above(min_gain)
    print(result)

In [ ]:
part_1(EXAMPLE, EXAMPLE_MIN_GAIN)

In [ ]:
part_1(INPUT, INPUT_MIN_GAIN)

In [ ]:
def part_2(input_file_name, min_gain):
    map = Map.build(input_file_name)
    map.find_race_track()
    map.find_cheats(max_nb_of_moves=20)
    result = map.count_cheats_above(min_gain)
    print(result)

In [ ]:
EXAMPLE_MIN_GAIN = 70
part_2(EXAMPLE, EXAMPLE_MIN_GAIN)

In [ ]:
part_2(INPUT, INPUT_MIN_GAIN)